<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/1-1.%20%EC%A6%9D%EB%B6%84%EB%90%9C%20F%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=a8a3c8003bba39daa90b7eef9b4bfaba260469dc57c3e151ebec6fadb04f2df2
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 77.2 MB/s 
     |████████████████████████████████| 3.0 MB 58.7 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4k4b6zda
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화    '답안 작성용 파일.csv'  'My Drive'
'Colab Notebooks'   df_F_inc.csv


In [18]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/df_F_inc.csv')

## 데이터 전처리

In [19]:
df['digit_3'].value_counts()

1    26904
7    12528
5    11152
3     5744
2     4724
6     4472
4     4360
0     3582
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [20]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['영업장에서 고객의뢰를 받아 내부전기공사', '4'], ['사업장에서 의뢰.요청받아 상수도배관공사', '3'], ['현장에서 고개의 요청에따라 주택건설', '0'], ['건축물내 기계,장비설치공사 공조기 설치', '3'], ['건물내 고객요구에 따라 인테리어공사', '5']]


## Train data & test data

In [21]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [22]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [23]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [24]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [26]:
data_train[0]

(array([   2,  885, 6873, 1306,  885, 4737, 6217, 6712, 6241, 2658, 7782,
        2874, 5474, 1043,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [30]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1034 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.171375274658203 train acc 0.046875
epoch 1 batch id 201 loss 1.2552531957626343 train acc 0.3714241293532338
epoch 1 batch id 401 loss 0.31942516565322876 train acc 0.656639650872818
epoch 1 batch id 601 loss 0.11991967260837555 train acc 0.7634671381031614
epoch 1 batch id 801 loss 0.1201990395784378 train acc 0.817103620474407
epoch 1 batch id 1001 loss 0.03594047948718071 train acc 0.8484172077922078
epoch 1 train acc 0.8525749516441006


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 29 test acc 0.9616501491901108


  0%|          | 0/1034 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.019400443881750107 train acc 0.984375
epoch 30 batch id 201 loss 0.0016431224066764116 train acc 0.9977456467661692
epoch 30 batch id 401 loss 0.000558879692107439 train acc 0.9974283042394015
epoch 30 batch id 601 loss 0.0036130594089627266 train acc 0.9976861480865225
epoch 30 batch id 801 loss 0.00020611716900020838 train acc 0.9975226279650437
epoch 30 batch id 1001 loss 0.00015661827637813985 train acc 0.9976429820179821
epoch 30 train acc 0.9976426499032882


  0%|          | 0/115 [00:00<?, ?it/s]

epoch 30 test acc 0.9615142796248934


In [31]:
#학습 모델 저장
torch.save(model, 'F_model_2.pt')

# 문장 test 해보기

In [32]:
#path='/content/'
#model = torch.load('dabunryu_model.pt')

model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [33]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)  
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        return np.argmax(logits)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 제출하기

In [38]:
ts=pd.read_csv('/content/gdrive/MyDrive/답안 작성용 파일.csv', encoding="euc-kr")

In [40]:
ts=ts[ts['digit_1']=='F']

In [42]:
ts=ts.reset_index(drop=True)
ts

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000065,F,NaN,NaN,사업장에서,계약을체결하여,토목공사
1,id_000100,F,NaN,NaN,배관 및 냉·난방 공사업,고객의 의뢰에 의해,공사
2,id_000116,F,NaN,NaN,현장에서,고객요청에의해,소방기계설비건설
3,id_000119,F,NaN,NaN,건설현장에서,고객요구에 따라,철근설치
4,id_000164,F,NaN,NaN,사업체에서,의뢰를받아,도로포장공사
...,...,...,...,...,...,...,...
5049,id_099933,F,NaN,NaN,사업장에서,"철,제딩기만들어","댐,수문,하수처리 장치공사"
5050,id_099938,F,NaN,NaN,현장에서,H빔으로 철골 용접,철골공사
5051,id_099946,F,NaN,NaN,사업체에서,의뢰를 받아,건물내 전기공사
5052,id_099987,F,NaN,NaN,영업장에서,공사수주,지붕판금공사.조립공사


In [43]:
ts['text']=ts['text_obj'].map(str)+" "+ts['text_mthd'].map(str)+" "+ts['text_deal'].map(str)

i=[]
for sentence in ts['text']:
    a=sentence.replace('nan', '')
    i.append(a)
ts['text']=pd.DataFrame(i)

ts=ts.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

ts

,AI_id,digit_1,digit_2,digit_3,text
0,id_000065,F,NaN,NaN,사업장에서 계약을체결하여 토목공사
1,id_000100,F,NaN,NaN,배관 및 냉·난방 공사업 고객의 의뢰에 의해 공사
2,id_000116,F,NaN,NaN,현장에서 고객요청에의해 소방기계설비건설
3,id_000119,F,NaN,NaN,건설현장에서 고객요구에 따라 철근설치
4,id_000164,F,NaN,NaN,사업체에서 의뢰를받아 도로포장공사
...,...,...,...,...,...
5049,id_099933,F,NaN,NaN,"사업장에서 철,제딩기만들어 댐,수문,하수처리 장치공사"
5050,id_099938,F,NaN,NaN,현장에서 H빔으로 철골 용접 철골공사
5051,id_099946,F,NaN,NaN,사업체에서 의뢰를 받아 건물내 전기공사
5052,id_099987,F,NaN,NaN,영업장에서 공사수주 지붕판금공사.조립공사


In [ ]:
ts1=ts[:]
result=[]
for sentence in ts1['text']:
  temp=predict(sentence)
  result.append(temp)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
a=pd.DataFrame(result)
ts1=ts1.reset_index()
result0=pd.concat([ts1, a], axis=1)
result0.to_csv('result_F.csv', index=False, encoding='utf-8-sig')